In [1]:
# Initial analysis of data and layout of code

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import requests
import os
import gzip
import shutil
import gdown
import subprocess
from math import radians, cos, sin, asin, sqrt

In [3]:
pd.set_option("display.max_columns", 100)

In [7]:
# Get Locations URL and Load into Pandas Dataframe
locations_url = "https://drive.google.com/file/d/1WOmj8wSpe8FDn_7opryh9tsng8ZqhAr1/view?usp=share_link"
reconstructed_locations_url = 'https://drive.google.com/uc?id=' + locations_url.split('/')[-2]
locations = pd.read_csv(reconstructed_locations_url)

cwd = os.getcwd()
locations.to_csv(cwd +'/data/locations.csv')

/var/folders/gh/2jk0_tgj2s5czzj043hknhb40000gn/T/ipykernel_29085/2977275755.py:4: DtypeWarning: Columns (11,12,14,20) have mixed types. Specify dtype option on import or set low_memory=False.
  locations = pd.read_csv(reconstructed_locations_url)


In [ ]:
display(locations)

In [417]:
# Cleaning Steps
# Remove Rows without f_lat & f_lon

# Initial research shows that these addresses are likely related to land without a building
# 200 Wesley Ave, 3963 Morrison Rd, etc. seem to be vacant land or lots

print('Original number of locations provided: ' + str(locations.shape[0]))
print('Number of locations with missing f_lat value: ' + str(locations['f_lat'].isna().sum()))
print('Number of locations with missing f_lon value: ' + str(locations['f_lon'].isna().sum()))

#locations = locations.dropna(subset=['f_lat','f_lon'])

#print('Total Number of locations after dropping NaN lat/long values: ' + str(locations.shape[0]))

Original number of locations provided: 156900
Number of locations with missing f_lat value: 7718
Number of locations with missing f_lon value: 7718


In [418]:
sum(locations['parcel_id'] != 'None')
# Potentially 114254 or 68% could be matched directly to a building

114254

In [8]:
# Get buildings data
# https://stackoverflow.com/questions/5710867/downloading-and-unzipping-a-zip-file-without-writing-to-disk
buildings_url = "https://drive.google.com/file/d/1qO86txHm82OqWbEEIEsaevF_tRFv4PhK/view?usp=share_link"
reconstructed_buildings_url = 'https://drive.google.com/uc?id=' + buildings_url.split('/')[-2]

cwd = os.getcwd()

resp = urlopen(reconstructed_buildings_url)
myzip = ZipFile(BytesIO(resp.read()))
myzip.extractall(cwd +'/data/')

# Read json
buildings = gpd.read_file(cwd +'/data/ms_hinds_buildings.json')
buildings_join_tbl = pd.read_csv(cwd +'/data/ms_hinds_buildings_join_table.csv')

In [ ]:
display(buildings)

In [ ]:
# Get parcels data
parcels_url = "https://drive.google.com/file/d/137bCzf0qEPLwKTgNxeAAjTs5MvyTlxWA/view?usp=share_link"
reconstructed_parcels_url = 'https://drive.google.com/uc?id=' + parcels_url.split('/')[-2]

# Use gdown to download large gz file: https://pypi.org/project/gdown/
output = cwd + '/data/ms_hinds_parcels.ndgeojson.gz'
gdown.download(reconstructed_parcels_url, output, quiet=False)

# Extract and save gzip file
gzip_filename = cwd + '/data/ms_hinds_parcels.ndgeojson.gz'
dest_filepath = cwd + '/data/ms_hinds_parcels.ndgeojson'
with gzip.open(gzip_filename, 'rb') as file:
        with open(dest_filepath, 'wb') as output_file:
            output_file.write(file.read())

parcels = gpd.read_file(cwd +'/data/ms_hinds_parcels.ndgeojson')   

# Method not needed: use geopandas to directly load file
# Original method was to use shell script and convert to geojson
# Use shell script to convert npgeojson to geojson
#subprocess.call(['sh', './convert_json.sh'])

# # Read json
# final_filepath = cwd + '/data/ms_hinds_parcels.geojson'
# parcels1 = gpd.read_file(final_filepath)

# # New method read as json, then normalize
# parcels = pd.read_json(cwd + '/data/ms_hinds_parcels.ndgeojson', lines=True)
# parcels = pd.json_normalize(parcels.properties)
# parcels.to_csv(cwd +'/data/parcels.csv')

In [7]:
# # Quick to test re-run
# cwd = os.getcwd()
# locations = pd.read_csv(cwd +'/data/locations.csv')
# buildings = gpd.read_file(cwd +'/data/ms_hinds_buildings.json')
# buildings_join_tbl = pd.read_csv(cwd +'/data/ms_hinds_buildings_join_table.csv')
# parcels = gpd.read_file(cwd +'/data/ms_hinds_parcels.ndgeojson')

/var/folders/gh/2jk0_tgj2s5czzj043hknhb40000gn/T/ipykernel_34714/1695776075.py:3: DtypeWarning: Columns (11,12,14,20) have mixed types. Specify dtype option on import or set low_memory=False.
  locations = pd.read_csv(cwd +'/data/locations.csv')


In [8]:
# Build combined parcel/bldg table
# Isolate from parcel: ll_uuid, address, szip
parcels_join = parcels[['ll_uuid', 'address', 'szip']]
# Confirm szip is 5 digit zip code to match f_ziplock
parcels_join['szip'] = parcels_join['szip'].str.replace("-.*","")
# Confirmed there were no duplicates here

/var/folders/gh/2jk0_tgj2s5czzj043hknhb40000gn/T/ipykernel_34714/1114220313.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  parcels_join['szip'] = parcels_join['szip'].str.replace("-.*","")
/var/folders/gh/2jk0_tgj2s5czzj043hknhb40000gn/T/ipykernel_34714/1114220313.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parcels_join['szip'] = parcels_join['szip'].str.replace("-.*","")


In [9]:
# Join to get ed_str_uuid and ed_bld_uuid
buildings_join_tbl = buildings_join_tbl.drop('geoid', axis=1)
parcels_join = parcels_join.merge(buildings_join_tbl, on='ll_uuid', how='left')

In [10]:
# Drop all ll_uuid without a str and bld uuid
parcels_join = parcels_join.dropna(subset=['ed_str_uuid','ed_bld_uuid'])

In [11]:
# Join to buildings on both uuids to get ed_lat/ed_lon
buildings_join = buildings[['ed_str_uuid', 'ed_bld_uuid', 'ed_lat', 'ed_lon']]
parcels_join = parcels_join.merge(buildings_join, on=['ed_str_uuid', 'ed_bld_uuid'], how='left')

In [58]:
# Combined Function
# Each location has a lamda function run against it to assign the best possible lat/long and distance from original
# 1. If location has a parcel_id and only one building on the parcel, the location will be assigned the building's lat/long
# 2. If location has a parcel_id and there are multiple buildings, calculate the smallest distance and assign that building's lat/long
# 3. If location has a parcel_id, but no point lat/long, assign the first building id associated with the parcel
# 4. If location has no parcel_id, look at nearest buildings and assign the closest building lat/long with the parcel
# 5. If location has no parcel_id or f_lat/f_long, no changes will be made


def hav_dist(lat1, long1, lat2, long2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

# def euc_dist(lat1, long1, lat2, long2):
#     return(np.sqrt(np.power(abs(lat1-lat2),2)+np.power(abs(long1-long2),2)))

def find_nearest(lat, lon, parcels_join_filt):
    distances = parcels_join_filt.apply(
        lambda row: hav_dist(lat, lon, row['ed_lat'], row['ed_lon']), 
        axis=1)
    return (parcels_join_filt.loc[distances.idxmin()]['ed_lat'], 
            parcels_join_filt.loc[distances.idxmin()]['ed_lon'],
            distances.min())

def get_updated_geo(ll_uuid,lat,lon):
    
    # parcels_join is loaded previously
    # Filter parcels_join for all buildings associated with parsel
    parcels_join_filt = parcels_join[parcels_join['ll_uuid'] == ll_uuid]
    
    if parcels_join_filt.shape[0] == 0:
        # Calculate closest building
        if np.isnan(lat):
            return(None)
        else:
            # Filter buildings lat long for different ranges to make for a smaller subset
            close_buildings = buildings[(buildings['ed_lat'] >= lat - 0.001) & 
                                        (buildings['ed_lat'] <= lat + 0.001) & 
                                        (buildings['ed_lon'] >= lon - 0.001) & 
                                        (buildings['ed_lon'] <= lon + 0.001)]
            if close_buildings.shape[0] == 0:
                # Refilter for a larger subset
                close_buildings = buildings[(buildings['ed_lat'] >= lat - 0.01) & 
                                            (buildings['ed_lat'] <= lat + 0.01) & 
                                            (buildings['ed_lon'] >= lon - 0.01) & 
                                            (buildings['ed_lon'] <= lon + 0.01)]
            if close_buildings.shape[0] == 0:
                # Refilter for a larger subset
                close_buildings = buildings[(buildings['ed_lat'] >= lat - 0.1) & 
                                            (buildings['ed_lat'] <= lat + 0.1) & 
                                            (buildings['ed_lon'] >= lon - 0.1) & 
                                            (buildings['ed_lon'] <= lon + 0.1)]
            if close_buildings.shape[0] == 0:
                # Refilter for a larger subset
                close_buildings = buildings[(buildings['ed_lat'] >= lat - 1.0) & 
                                            (buildings['ed_lat'] <= lat + 1.0) & 
                                            (buildings['ed_lon'] >= lon - 1.0) & 
                                            (buildings['ed_lon'] <= lon + 1.0)]
            if close_buildings.shape[0] == 0:
                return(None)
            else:
                return(find_nearest(lat,lon,close_buildings))
        
    elif parcels_join_filt.shape[0] == 1:
        # There is only one building associated with this parsel
        # Return lat/long of this building as the updated coordinates
        if np.isnan(lat):
            distance = 0
        else:
            distance = hav_dist(lat, lon, parcels_join_filt.ed_lat, parcels_join_filt.ed_lon)
        return(parcels_join_filt.ed_lat.iloc[0], parcels_join_filt.ed_lon.iloc[0], distance)
    
    else:
        # There are multiple buildings associated with this matching parsel
        # Calculate the haversine distance between the location and each building
        # Return the closest building
        if np.isnan(lat):
            parcels_join_filt = parcels_join_filt.groupby("ll_uuid").first()
            distance = 0
            return(parcels_join_filt.ed_lat.iloc[0], parcels_join_filt.ed_lon.iloc[0], distance)
        else:
            return(find_nearest(lat,lon,parcels_join_filt))

In [59]:
%%time

# This will run and take around 30 minutes
buildings = buildings[['ed_str_uuid', 'ed_bld_uuid', 'ed_lat', 'ed_lon']]
locations['updated_geo'] = locations.apply(lambda x: get_updated_geo(x.parcel_id, x.f_lat, x.f_lon), axis=1)
locations.to_csv(cwd +'/data/locations_updated.csv')

CPU times: user 27min 31s, sys: 30.1 s, total: 28min 1s
Wall time: 29min 37s


In [56]:
# TESTING
# locations2 = locations.sample(n=1000)
# buildings = buildings[['ed_str_uuid', 'ed_bld_uuid', 'ed_lat', 'ed_lon']]
# locations2 = locations[locations['parcel_id'] == '6adfe635-0a7a-42ea-83ff-cda265bd077e']
# locations2['updated_geo'] = locations2.apply(lambda x: get_updated_geo(x.parcel_id, x.f_lat, x.f_lon), axis=1)

In [413]:
# TESTING
# locations2.to_csv(cwd +'/data/locations_testing_sample.csv')

In [ ]:
locations_updated = pd.read_csv(cwd +'/data/locations_updated.csv')

# Split and clean updated information
locations_updated['updated_lat'] = locations_updated['updated_geo'].str.split(',', expand = True)[0]
locations_updated['updated_lon'] = locations_updated['updated_geo'].str.split(',', expand = True)[1]
locations_updated['distance_km'] = locations_updated['updated_geo'].str.split(',', expand = True)[2]

locations_updated['updated_lat'] = locations_updated['updated_lat'].str.replace("(","")
locations_updated['distance_km'] = locations_updated['distance_km'].str.replace(")","")

locations_updated['updated_lat'] = pd.to_numeric(locations_updated['updated_lat'])
locations_updated['updated_lon'] = pd.to_numeric(locations_updated['updated_lon'])
locations_updated['distance_km'] = pd.to_numeric(locations_updated['distance_km'])

In [ ]:
# Deliverable 1: Data table: with improved geo-precision of address locations, 
# ideally matching each address to the correct building with the address’s coords updated to building rooftop.

# Save Locations updated with new lat/long/distance from original point
locations_updated.to_csv(cwd +'/data/final.csv')

In [ ]:
# Deliverable 2: Visualization of locations BEFORE v.s. AFTER on top of building polygons / parcel polygons
# See other file

In [ ]:
# Deliverable 3: Metrics:
# 1. On average, how far are original geolocation moved

#locations_updated['distance_km'] = pd.to_numeric(locations_updated['distance_km'])
avg_distance_moved = locations_updated['distance_km'].mean()
min_distance_moved = locations_updated['distance_km'].min()
max_distance_moved = locations_updated['distance_km'].max()

# 2. How many points with too little information to move anywhere?
# Only points without an original lat/long and a parcel without a matching building could not be moved
total_locations = locations_updated.shape[0]
change_locations = locations_updated.dropna(subset=['updated_geo']).shape[0]
no_change_locations = total_locations - change_locations

# 3. What else (other data) can we use to enhance the accuracy?

In [ ]:
print('Average distance (km) points moved: ' + str(avg_distance_moved))
print('Smallest distance (km) moved: ' + str(min_distance_moved))
print('Largest distance (km) moved: ' + str(max_distance_moved))
print('Number of Locations without an updated geolocation: ' + str(no_change_locations))

In [ ]:
data = [[avg_distance_moved, min_distance_moved, max_distance_moved, no_change_locations]]
df = pd.DataFrame(data, columns=['avg_distance_moved', 'min_distance_moved', 'max_distance_moved', 'no_change_locations'])

display(df)